In [33]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import scipy
import sklearn
import random as rd
from IPython.display import display
import os 
from pathlib import Path
import shutil 


### IF WE HAVE THE CK+ DATABASE, WE CAN DO THIS CODE IN THE NEXTS CELLS TO CREATE A MINI DATABASE TO START WITH THE FEATURES

In [5]:
 os.listdir('../../ck+/Landmarks' )

['S125',
 'S133',
 'S078',
 'S117',
 'S029',
 'S035',
 'S109',
 'S079',
 'S076',
 'S099',
 'S051',
 'S156',
 'S148',
 'S506',
 'S045',
 'S077',
 'S502',
 'S052',
 'S090',
 'S014',
 'S095',
 'S104',
 'S083',
 'S130',
 'S127',
 'S059',
 'S151',
 'S136',
 'S073',
 'S085',
 'S056',
 'S069',
 'S114',
 'S138',
 'S071',
 'S137',
 'S067',
 'S110',
 'S091',
 'S503',
 'S096',
 'S070',
 'S121',
 'S046',
 'S158',
 'S139',
 'S097',
 'S010',
 'S058',
 'S037',
 'S092',
 'S102',
 'S120',
 'S504',
 'S063',
 'S106',
 'S061',
 'S066',
 'S086',
 'S107',
 'S135',
 'S055',
 'S112',
 'S501',
 'S074',
 'S131',
 'S084',
 'S101',
 'S054',
 'S072',
 'S134',
 'S126',
 'S089',
 'S999',
 'S155',
 'S119',
 'S064',
 'S129',
 'S103',
 'S124',
 'S080',
 'S075',
 'S154',
 'S062',
 'S105',
 'S113',
 'S111',
 'S082',
 'S093',
 'S087',
 'S053',
 'S088',
 'S147',
 'S034',
 'S060',
 'S065',
 'S081',
 'S042',
 'S032',
 'S108',
 'S160',
 'S005',
 'S057',
 'S115',
 'S157',
 'S022',
 'S132',
 'S116',
 'S044',
 'S149',
 'S026',
 

##Elegimos la cantidad de sujetos para la extraccion (=11) de ck al azar. Luego eligiremos una con interseccion 0 para la evaluacio (y una parte mas chica con interseccion 0 con las otras 2 papra validacion)

In [62]:
Emotion_subjects = os.listdir('../../ck+/Emotion_labels/Emotion' )
len(Emotion_subjects)
images_subjects = os.listdir('../../ck+/extended-cohn-kanade-images/cohn-kanade-images' )
len(images_subjects)
facts_subjects =  os.listdir('../../ck+/FACS_labels/FACS' )
len(facts_subjects)
landmarks_subjects =  os.listdir('../../ck+/Landmarks' )
len(landmarks_subjects)



123

In [63]:
def choose_random_subjects(main_dir, local_dir, amount):
    subjects = os.listdir(main_dir + local_dir )
    random_subjects = np.random.choice(subjects, amount,replace=False )
    
    return random_subjects
    
subject_number =choose_random_subjects('../../ck+/','extended-cohn-kanade-images/cohn-kanade-images', 11)

subject_number

array(['S091', 'S067', 'S999', 'S042', 'S112', 'S080', 'S094', 'S071',
       'S011', 'S154', 'S127'], dtype='<U4')

In [64]:
def do_assigned_dir(orsrc, ordst, subjects):
    for number in subjects:
        src = orsrc + number
        dst = ordst + number
        destination = shutil.copytree(src, dst)
        print(destination)
    return 0

In [65]:
do_assigned_dir('../../ck+/extended-cohn-kanade-images/cohn-kanade-images/', '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/', subject_number )
do_assigned_dir('../../ck+/Emotion_labels/Emotion/', '../ck+copy/Emotion_labels/Emotion/', subject_number )
do_assigned_dir('../../ck+/FACS_labels/FACS/', '../ck+copy/FACS_labels/FACS/', subject_number )
do_assigned_dir('../../ck+/Landmarks/', '../ck+copy/Landmarks/', subject_number )

../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S091
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S067
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S999
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S042
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S112
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S080
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S094
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S071
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S011
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S154
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S127
../ck+copy/Emotion_labels/Emotion/S091
../ck+copy/Emotion_labels/Emotion/S067
../ck+copy/Emotion_labels/Emotion/S999
../ck+copy/Emotion_labels/Emotion/S042
../ck+copy/Emotion_labels/Emotion/S112
../ck+copy/Emotion_labels/Emotion/S080
../ck+copy/Emotion_labels/Emotion/S094
../ck+copy/Emotion_labels/Emotion/

0

#### Aqui sacamos las carpetas que no tienen emocion definida o es contempt, que no usamos

In [66]:
deletefolders = []
for number in subject_number:
  
    src ='../ck+copy/Emotion_labels/Emotion/'  + number
    emotion_folders = np.sort(os.listdir(src))
    for number_emo in emotion_folders:
        emotion = src  +'/'+ number_emo             
        if not os.listdir(emotion):
            deletefolders.append([number, number_emo])
        else:
            filelabel = os.listdir(emotion)[0]
            with open(emotion+ '/'+filelabel) as f:
                line = float(f.readline())
            if (line ==2.0):
                deletefolders.append([number,number_emo])
                # 2 == contempt emotion
            
   

In [67]:
for folder in deletefolders:
    image = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/'+ folder[0]+'/' + folder[1]
    facs =  '../ck+copy/FACS_labels/FACS/'+ folder[0]+'/' + folder[1]
    emotion = '../ck+copy/Emotion_labels/Emotion/' + folder[0]+ '/' + folder[1]
    landmark = '../ck+copy/Landmarks/' + folder[0]+ '/' + folder[1]
    shutil.rmtree(image)
    shutil.rmtree(facs)
    shutil.rmtree(emotion)
    shutil.rmtree(landmark)

In [127]:
subjects_landmarks = '../ck+copy/Landmarks/'
sorted_subjects =   np.sort(os.listdir('../ck+copy/Landmarks/'))  
one_subject_landmarks = '../ck+copy/Landmarks/' + 'S064/'
sorted_list = np.sort(os.listdir('../ck+copy/Landmarks/' + 'S064/')) #creo que este sort se puede quitar


    
distances_per_subject = []
for emotion in sorted_list:
    matrix_landmark_per_frame = []
    subject_landmark_emotion = one_subject_landmarks + str(emotion)    
    for i in range(0, len(os.listdir(subject_landmark_emotion))):
        data = pd.read_csv('../ck+copy/Landmarks/' + 'S064/' + str(emotion)+'/'+'S064_'+str(emotion)+'_'+ str(i+1).zfill(8)+'_landmarks.txt',  header=None)
        data.columns =['landmarks']
        quantity_of_landmarks = data.size #68
        data = data['landmarks'].str.split('   ', expand=True)
        data.columns =['landmarks', 'landmarkX', 'landmarkY']
        data =data.drop(columns=['landmarks'])
        landmarkXY=data['landmarkX'].append(data['landmarkY'], ignore_index=True)
        matrix_landmark_per_frame.append(landmarkXY_array)
    frame0 = matrix_landmark_per_frame[0]
    distance_landmark_per_frame = matrix_landmark_per_frame
    for i in range(0, len(distance_landmark_per_frame)):
        distance_landmark_per_frame[i] = distance_landmark_per_frame[i] - frame0
    distances_per_subject.append(distance_landmark_per_frame)
